In [1]:
putStrLn "你好世界！"

你好世界！

In [18]:
do
  putStrLn "再见宇宙！"
  putStrLn "开玩笑的"

再见宇宙！
开玩笑的

In [2]:
import Data.Char

In [3]:
ord '我'

25105

In [4]:
putStrLn [chr 25105]

我

In [5]:
import System.Random

In [6]:
-- generate 0 or 1
randomRIO (0, 1)

1

In [7]:
randomRIO (0x4e00, 0x9fff)

21092

In [8]:
import Control.Monad

In [16]:
fmap ((: []) . chr) (randomRIO (0x4e00, 0x9fff)) >>= putStrLn

钨

In [26]:
do
  ord <- randomRIO (0x4e00, 0x9fff)
  let hanzi = (: []) $ chr ord
  putStrLn hanzi

辜

In [33]:
let randomHanzi () = fmap chr $ randomRIO (0x4e00, 0x9fff)
:t randomHanzi  

randomHanzi :: () -> IO Char

In [34]:
:t foldl

foldl :: forall (t :: * -> *) b a. Foldable t => (b -> a -> b) -> b -> t a -> b

In [ ]:
let doN n _fn when n <= 0 = []
let doN n fn = (fn ()) : doN (n - 1) fn